### Importing libraries

In [1]:
import numpy as np
import pandas as pd
import random
import math

from sklearn import preprocessing # normalization library


### Robot Data
Pioneer 3dx

In [2]:
LMr=381e-3
R=195e-3

v_max= 1
v_min=0.3

Lt_max=1.2
Lt_min=0.3

Mean Velocity

In [3]:
v_mean=(v_max+v_min)/2
print(v_mean)

0.65


### Obstacle circle 

Sensor range data: $[dmin_1\quad dmax_1]\quad m$ 

Heading angle data: $[-\pi\quad\pi]\quad rad$ 


In [4]:
min_d1=0.3
max_d1=1

In [5]:
# Sensor Data gen 
Obs_Data=[round(random.uniform(min_d1, max_d1),4) for _ in range(3000*3)]
Obs_Data_M=np.array(Obs_Data).reshape(3000,3)

print('Raw data :',Obs_Data_M[:2,:])

normalizer=preprocessing.MinMaxScaler()
Obs_Data_Norm=normalizer.fit_transform(Obs_Data_M)
print('Normalized data :',Obs_Data_Norm[:2,:])


Raw data : [[0.7291 0.9099 0.4465]
 [0.7621 0.47   0.9277]]
Normalized data : [[0.61308758 0.87119371 0.20909611]
 [0.66023718 0.24231594 0.89731121]]


In [6]:
# HA data gen

HA_Data=[round(random.uniform(-math.pi, math.pi),4) for _ in range(3000)]
HA_Data=np.array(HA_Data).reshape(3000,1)
print('Angle data :',HA_Data[:2])

Angle data : [[ 2.9772]
 [-1.461 ]]


Stack Normal Data

In [7]:


C1_Normal_Data=np.hstack((Obs_Data_Norm,HA_Data))
print('Normal array :',C1_Normal_Data[:2])

Normal array : [[ 0.61308758  0.87119371  0.20909611  2.9772    ]
 [ 0.66023718  0.24231594  0.89731121 -1.461     ]]


### Special Data for small angles and near obstacle

In [8]:
Obs_Data_Near=[round(random.uniform(min_d1,max_d1-0.6),4) for _ in range(1000*3)]
Obs_Data_M_Near=np.array(Obs_Data_Near).reshape(1000,3)

print('Raw data :',Obs_Data_M_Near[:2,:])


Norm_Near=normalizer.fit(Obs_Data_M)
Obs_Data_Near=Norm_Near.transform(Obs_Data_M_Near)

print('Normalized data :',Obs_Data_Near[:2,:])


Raw data : [[0.3483 0.3621 0.3289]
 [0.3427 0.3889 0.391 ]]
Normalized data : [[0.06900986 0.0880629  0.04090389]
 [0.06100872 0.12637598 0.12971968]]


Small angles

In [9]:
Small_Theta_Data=[round(random.uniform(-0.5, 0.5),4) for _ in range(1000)]
Small_Theta_Array=np.array(Small_Theta_Data).reshape(1000,1)
print('Angle data :',Small_Theta_Array[:2])

Angle data : [[0.0087]
 [0.2717]]


Stack small

In [10]:
C1_Small_Data=np.hstack((Obs_Data_Near,Small_Theta_Array))
print('Small array :',C1_Small_Data[:2])

Small array : [[0.06900986 0.0880629  0.04090389 0.0087    ]
 [0.06100872 0.12637598 0.12971968 0.2717    ]]


Stack all and Randomize

In [11]:
# Stack All

St_train_C1=np.vstack((C1_Normal_Data,C1_Small_Data))
np.random.shuffle(St_train_C1)
print(St_train_C1[:2])



[[0.33876268 0.89220872 0.30005721 0.9721    ]
 [0.53121875 0.22544675 0.60440503 2.1746    ]]


Make dataframe

In [12]:
Df_C1=pd.DataFrame(St_train_C1, columns=['LS','FS','RS','Thetad'])
Df_C1.sample(6)

,LS,FS,RS,Thetad
1874,0.688384,0.105075,0.373999,-2.3902
2756,0.167881,0.717227,0.754291,-2.8133
3101,0.071725,0.533953,0.104405,-3.1241
2746,0.324475,0.989421,0.620137,1.1373
2345,0.398914,0.965690,0.437357,-3.0692
3505,0.969281,0.081201,0.963244,-0.8700


### Inverse Kinematic Model

$$\omega_{R/L}=\left( 1\pm\frac{L_{MR}}{2L_T}\right)\frac{v}{R}$$

Gain near extremes normalized

In [13]:
def gain_near_extremes_normalized(x):
     return 4 * x * (1 - x)

In [14]:
def gain_near_zero(x, scale=10):
     gain = np.exp(-scale * x)
     normalized_gain = (gain - np.exp(-scale)) / (1 - np.exp(-scale))
     return normalized_gain
    

First Case: Left sensor nearest, objective angle +

In [15]:
odp=1
theta_d=3


S=-1
factor=round(gain_near_extremes_normalized(odp),4)
print('Factor : ',factor)
Lt_p=round(((odp)*(Lt_max-Lt_min)+Lt_min),4)
print('Ltp: ',Lt_p)
V_p=round(v_max-(factor*(v_mean/2)),4)
print('vel: ',V_p)
if odp<0.2 and theta_d<0.5:
    fth=round(gain_near_zero(theta_d,scale=10),4)
    print('Factor theta:',fth)
    theta_d=round(fth*math.pi,4)
else:
    theta_d=theta_d
print('Tetha sel: ',theta_d)
W_R=round((1+((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R),4)
W_L=round((1-((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R),4)
print('Right Wheel vel: ',W_R)
print('Left Wheel vel: ',W_L)


def Case_1_R(odp,theta_d):
    S=-1
    factor=round(gain_near_extremes_normalized(odp),4)
    Lt_p=round(((odp)*(Lt_max-Lt_min)+Lt_min),4)
    V_p=round(v_max-(factor*(v_mean/2)),4)
    if odp<0.2 and theta_d<0.5:
        fth=round(gain_near_zero(theta_d,scale=10),4)
        theta_d=round(fth*math.pi,4)
    else:
        theta_d=theta_d
    return round((1+((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R),4)

    
def Case_1_L(odp,theta_d):
    S=-1
    factor=round(gain_near_extremes_normalized(odp),4)
    Lt_p=round(((odp)*(Lt_max-Lt_min)+Lt_min),4)
    V_p=round(v_max-(factor*(v_mean/2)),4)
    if odp<0.2 and theta_d<0.5:
        fth=round(gain_near_zero(theta_d,scale=10),4)
        theta_d=round(fth*math.pi,4)
    else:
        theta_d=theta_d
    return round((1-((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R),4)

    

Factor :  0
Ltp:  1.2
vel:  1.0
Tetha sel:  3
Right Wheel vel:  2.6859
Left Wheel vel:  7.5705


Second Case: Left sensor nearest, objective angle -

In [16]:

odp=1
theta_d=-2
S=1
factor=round(gain_near_extremes_normalized(odp),4)
print('Factor : ',factor)

Lt_p=round(((odp)*(Lt_max-Lt_min)+Lt_min),4)
print('Ltp: ',Lt_p)
V_p=round(v_max-(factor*(v_mean/2)),4)
print('vel: ',V_p)

if odp<0.2 and theta_d>-0.5 and theta_d<0:
    fth=round(gain_near_zero(-1*theta_d),4)
    print('Factor theta:',fth)
    theta_d=fth*-math.pi
else:
    theta_d=theta_d
    
cf=math.pi/4
    
print('Tetha sel: ',theta_d)
W_R=round((1+((LMr*(theta_d+cf)*S)/(2*Lt_p)))*(V_p/R),4)
W_L=round((1-((LMr*(theta_d+cf)*S)/(2*Lt_p)))*(V_p/R),4)
print('Right Wheel vel: ',W_R)
print('Left Wheel vel: ',W_L)

def Case_2_R(odp,theta_d):
    S=1
    factor=round(gain_near_extremes_normalized(odp),4)
    Lt_p=round(((odp)*(Lt_max-Lt_min)+Lt_min),4)
    V_p=round(v_max-(factor*(v_mean/2)),4)
    if odp<0.2 and theta_d>-0.5 and theta_d<0:
        fth=round(gain_near_zero(-1*theta_d),4)
        theta_d=fth*-math.pi
    else:
        theta_d=theta_d   
    cf=math.pi/4   
    return round((1+((LMr*(theta_d+cf)*S)/(2*Lt_p)))*(V_p/R),4)
    
def Case_2_L(odp,theta_d):
    S=1
    factor=round(gain_near_extremes_normalized(odp),4)
    Lt_p=round(((odp)*(Lt_max-Lt_min)+Lt_min),4)
    V_p=round(v_max-(factor*(v_mean/2)),4)
    if odp<0.2 and theta_d>-0.5 and theta_d<0:
        fth=round(gain_near_zero(-1*theta_d),4)
        theta_d=fth*-math.pi
    else:
        theta_d=theta_d   
    cf=math.pi/4   
    return round((1-((LMr*(theta_d+cf)*S)/(2*Lt_p)))*(V_p/R),4)

Factor :  0
Ltp:  1.2
vel:  1.0
Tetha sel:  -2
Right Wheel vel:  4.1394
Left Wheel vel:  6.117


Third case: Right sensor nearest, objective angle +

In [17]:
odp=0.1
theta_d=0.15
S=1
factor=round(gain_near_extremes_normalized(odp),4)
print('Factor : ',factor)

Lt_p=round(((odp)*(Lt_max-Lt_min)+Lt_min),4)
print('Ltp: ',Lt_p)
V_p=round(v_max-(factor*(v_mean/2)),4)
print('vel: ',V_p)

if odp<0.2 and theta_d<0.5 and theta_d>0:
    fth=round(gain_near_zero(theta_d),4)
    print('Factor theta:',fth)
    theta_d=round(fth*-math.pi)
else:
    theta_d=theta_d
cf=math.pi/4
    
print('Tetha sel: ',theta_d)
W_R=round((1+((LMr*(theta_d-cf)*S)/(2*Lt_p)))*(V_p/R),4)
W_L=round((1-((LMr*(theta_d-cf)*S)/(2*Lt_p)))*(V_p/R),4)
print('Right Wheel vel: ',W_R)
print('Left Wheel vel: ',W_L)

def Case_3_R(odp,theta_d):
    S=1
    factor=round(gain_near_extremes_normalized(odp),4)
    Lt_p=round(((odp)*(Lt_max-Lt_min)+Lt_min),4)
    V_p=round(v_max-(factor*(v_mean/2)),4)
    if odp<0.2 and theta_d<0.5 and theta_d>0:
        fth=round(gain_near_zero(theta_d),4)
        theta_d=round(fth*-math.pi)
    else:
        theta_d=theta_d
    cf=math.pi/4
    return round((1+((LMr*(theta_d-cf)*S)/(2*Lt_p)))*(V_p/R),4)

def Case_3_L(odp,theta_d):
    S=1
    factor=round(gain_near_extremes_normalized(odp),4)
    Lt_p=round(((odp)*(Lt_max-Lt_min)+Lt_min),4)
    V_p=round(v_max-(factor*(v_mean/2)),4)
    if odp<0.2 and theta_d<0.5 and theta_d>0:
        fth=round(gain_near_zero(theta_d),4)
        theta_d=round(fth*-math.pi)
    else:
        theta_d=theta_d
    cf=math.pi/4
    return round((1-((LMr*(theta_d-cf)*S)/(2*Lt_p)))*(V_p/R),4)

Factor :  0.36
Ltp:  0.39
vel:  0.883
Factor theta: 0.2231
Tetha sel:  -1
Right Wheel vel:  0.5792
Left Wheel vel:  8.4772


Fourth case: Right sensor nearest, objective angle -

In [18]:
odp=1
theta_d=0
S=-1

factor=round(gain_near_extremes_normalized(odp),4)

Lt_p=round(((odp)*(Lt_max-Lt_min)+Lt_min),4)
print('Ltp: ',Lt_p)
V_p=round(v_max-(factor*(v_mean/2)),4)
print('vel: ',V_p)

if odp<0.2 and theta_d>-0.5 and theta_d<0:
    fth=round(gain_near_zero(-1*theta_d),4)
    print('Factor theta:',fth)
    theta_d=round(fth*-math.pi,4)
else:
    theta_d=theta_d


print('Tetha sel: ',theta_d)

W_R=round((1+((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R),4)
W_L=round((1-((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R),4)
print('Right Wheel vel: ',W_R)
print('Left Wheel vel: ',W_L)

def Case_4_R(odp,theta_d):
    S=-1
    factor=round(gain_near_extremes_normalized(odp),4)
    Lt_p=round(((odp)*(Lt_max-Lt_min)+Lt_min),4)
    V_p=round(v_max-(factor*(v_mean/2)),4)
    if odp<0.2 and theta_d>-0.5 and theta_d<0:
        fth=round(gain_near_zero(-1*theta_d),4)
        theta_d=round(fth*-math.pi,4)
    else:
        theta_d=theta_d
        
    return round((1+((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R),4)


def Case_4_L(odp,theta_d):
    S=-1
    factor=round(gain_near_extremes_normalized(odp),4)
    Lt_p=round(((odp)*(Lt_max-Lt_min)+Lt_min),4)
    V_p=round(v_max-(factor*(v_mean/2)),4)
    if odp<0.2 and theta_d>-0.5 and theta_d<0:
        fth=round(gain_near_zero(-1*theta_d),4)
        theta_d=round(fth*-math.pi,4)
    else:
        theta_d=theta_d
        
    return round((1-((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R),4)

Ltp:  1.2
vel:  1.0
Tetha sel:  0
Right Wheel vel:  5.1282
Left Wheel vel:  5.1282


Fifth case: Front sensor nearest, objective angle +

In [19]:
odp=0.3
theta_d=3
S=-1

factor=round(gain_near_extremes_normalized(odp),4)
print('Factor : ',factor)


Lt_p=round(((odp)*(Lt_max-Lt_min)+Lt_min),4)
print('Ltp: ',Lt_p)
V_p=round(v_max-(factor*(v_mean/2)),4)
print('vel: ',V_p)

if odp<0.2 and theta_d<0.5:
    fth=round(gain_near_zero(theta_d,scale=10),4)
    print('Factor theta:',fth)
    theta_d=round(fth*math.pi,4)
else:
    theta_d=theta_d

print('Tetha sel: ',theta_d)
W_R=round((1+((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R),4)
W_L=round((1-((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R),4)
print('Right Wheel vel: ',W_R)
print('Left Wheel vel: ',W_L)


def Case_5_R(odp,theta_d):
    S=-1
    factor=round(gain_near_extremes_normalized(odp),4)
    V_p=round(v_max-(factor*(v_mean/2)),4)
    if odp<0.2 and theta_d<0.5:
        fth=round(gain_near_zero(theta_d,scale=10),4)
        theta_d=round(fth*math.pi,4)
    else:
        theta_d=theta_d
    return round((1+((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R),4)


def Case_5_L(odp,theta_d):
    S=-1
    factor=round(gain_near_extremes_normalized(odp),4)
    V_p=round(v_max-(factor*(v_mean/2)),4)
    if odp<0.2 and theta_d<0.5:
        fth=round(gain_near_zero(theta_d,scale=10),4)
        theta_d=round(fth*math.pi,4)
    else:
        theta_d=theta_d
    return round((1-((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R),4)

Factor :  0.84
Ltp:  0.57
vel:  0.727
Tetha sel:  3
Right Wheel vel:  -0.0098
Left Wheel vel:  7.4662


Sixth case: Front sensor nearest, objective angle -

In [20]:
odp=0.1
theta_d=-0.1
S=-1

factor=round(gain_near_extremes_normalized(odp),4)
print('Factor : ',factor)


Lt_p=round(((odp)*(Lt_max-Lt_min)+Lt_min),4)
print('Ltp: ',Lt_p)
V_p=round(v_max-(factor*(v_mean/2)),4)
print('vel: ',V_p)

if odp<0.2 and theta_d>-0.5 and theta_d<0:
    fth=round(gain_near_zero(-1*theta_d),4)
    print('Factor theta:',fth)
    theta_d=round(fth*-math.pi,4)
else:
    theta_d=theta_d


print('Tetha sel: ',theta_d)

W_R=round((1+((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R),4)
W_L=round((1-((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R),4)
print('Right Wheel vel: ',W_R)
print('Left Wheel vel: ',W_L)


def Case_6_R(odp,theta_d):
    S=-1
    factor=round(gain_near_extremes_normalized(odp),4)
    Lt_p=round(((odp)*(Lt_max-Lt_min)+Lt_min),4)
    V_p=round(v_max-(factor*(v_mean/2)),4)
    if odp<0.2 and theta_d>-0.5 and theta_d<0:
        fth=round(gain_near_zero(-1*theta_d),4)
        theta_d=round(fth*-math.pi,4)
    else:
        theta_d=theta_d
    return round((1+((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R),4)


def Case_6_L(odp,theta_d):
    S=-1
    factor=round(gain_near_extremes_normalized(odp),4)
    Lt_p=round(((odp)*(Lt_max-Lt_min)+Lt_min),4)
    V_p=round(v_max-(factor*(v_mean/2)),4)
    if odp<0.2 and theta_d>-0.5 and theta_d<0:
        fth=round(gain_near_zero(-1*theta_d),4)
        theta_d=round(fth*-math.pi,4)
    else:
        theta_d=theta_d
    return round((1-((LMr*theta_d*S)/(2*Lt_p)))*(V_p/R),4)

Factor :  0.36
Ltp:  0.39
vel:  0.883
Factor theta: 0.3679
Tetha sel:  -1.1558
Right Wheel vel:  7.0847
Left Wheel vel:  1.9717


### Dataset Creation

In [21]:
Df_C1['Min Value']=Df_C1[['LS','FS','RS']].min(axis=1)
Df_C1.sample(6)


,LS,FS,RS,Thetad,Min Value
761,0.710673,0.736669,0.779748,1.0909,0.710673
147,0.189313,0.210436,0.372140,2.5507,0.189313
2940,0.207601,0.912938,0.924056,1.4119,0.207601
88,0.004286,0.120372,0.032752,0.1475,0.004286
3195,0.792256,0.607863,0.986842,-3.0517,0.607863
677,0.283612,0.053038,0.677346,-0.7324,0.053038


### Left Wheel Evaluation

In [22]:
def lw_eval(df):
    if df['Min Value']== df['LS'] and df['Thetad']>0:
       if df['Thetad']==0:
          df['Thetad']=0.001
       return Case_1_L(df['Min Value'],df['Thetad']) 
    elif df['Min Value']== df['LS'] and df['Thetad']<0:
       if df['Thetad']==0:
           df['Thetad']=0.001
       return Case_2_L(df['Min Value'],df['Thetad']) 
    elif df['Min Value']== df['RS'] and df['Thetad']>0:
       if df['Thetad']==0:
           df['Thetad']=0.001
       return Case_3_L(df['Min Value'],df['Thetad']) 
    elif df['Min Value']== df['RS'] and df['Thetad']<0:
       if df['Thetad']==0:
           df['Thetad']=0.001
       return Case_4_L(df['Min Value'],df['Thetad']) 
    elif df['Min Value']== df['FS'] and df['Thetad']>0:
       if df['Thetad']==0:
           df['Thetad']=0.001
       return Case_5_L(df['Min Value'],df['Thetad']) 
    elif df['Min Value']== df['FS'] and df['Thetad']<0:
       if df['Thetad']==0:
           df['Thetad']=0.001
       return Case_6_L(df['Min Value'],df['Thetad']) 
    
    
    
def rw_eval(df):
      if df['Min Value']== df['LS'] and df['Thetad']>0:
       if df['Thetad']==0:
          df['Thetad']=0.001
       return Case_1_R(df['Min Value'],df['Thetad']) 
      elif df['Min Value']== df['LS'] and df['Thetad']<0:
       if df['Thetad']==0:
           df['Thetad']=0.001
       return Case_2_R(df['Min Value'],df['Thetad']) 
      elif df['Min Value']== df['RS'] and df['Thetad']>0:
       if df['Thetad']==0:
           df['Thetad']=0.001
       return Case_3_R(df['Min Value'],df['Thetad']) 
      elif df['Min Value']== df['RS'] and df['Thetad']<0:
       if df['Thetad']==0:
           df['Thetad']=0.001
       return Case_4_R(df['Min Value'],df['Thetad']) 
      elif df['Min Value']== df['FS'] and df['Thetad']>0:
       if df['Thetad']==0:
           df['Thetad']=0.001
       return Case_5_R(df['Min Value'],df['Thetad']) 
      elif df['Min Value']== df['FS'] and df['Thetad']<0:
       if df['Thetad']==0:
           df['Thetad']=0.001
       return Case_6_R(df['Min Value'],df['Thetad']) 
    

Df_C1=Df_C1.assign(vel_L=Df_C1.apply(lw_eval,axis=1))
Df_C1=Df_C1.assign(vel_R=Df_C1.apply(rw_eval,axis=1))

Df_C1.sample(6)



        

,LS,FS,RS,Thetad,Min Value,vel_L,vel_R
1926,0.032576,0.061615,0.069651,-0.3447,0.032576,2.9677,6.8682
1288,0.098014,0.013581,0.069508,-0.4817,0.013581,4.9618,5.1161
1059,0.005715,0.354253,0.752860,2.1465,0.005715,11.9124,-1.7319
1953,0.703100,0.114939,0.328518,-0.4226,0.114939,4.3538,4.5467
2219,0.810401,0.350822,0.889445,2.2906,0.350822,7.6486,-0.4291
2458,0.166452,0.468049,0.041905,-2.8995,0.041905,-3.0895,12.8105


Train Dataframe

In [23]:
Df_C1_Comp=Df_C1[['LS','FS','RS','Thetad','vel_L','vel_R']]
Df_C1_Comp.sample(6)

,LS,FS,RS,Thetad,vel_L,vel_R
1453,0.759251,0.033310,0.457380,-0.3038,4.4865,5.3402
286,0.272610,0.456040,0.429062,-0.8942,3.9508,3.6615
2235,0.088441,0.047748,0.194365,-2.5142,-1.9125,11.5628
859,0.532362,0.597427,0.261013,1.8975,2.3207,5.3645
3879,0.098300,0.063188,0.032752,0.3794,7.1496,2.6843
745,0.745535,0.498213,0.672054,-1.3173,2.3009,4.6222


In [24]:
from sklearn.model_selection import train_test_split

train_C1, val_C1 = train_test_split(Df_C1_Comp, test_size=0.2, random_state=42)

Extract vel L  to train and val

In [25]:
train_C1_vl=train_C1[['LS','FS','RS','Thetad','vel_L']]
train_C1_vl.sample(6)

,LS,FS,RS,Thetad,vel_L
518,0.768824,0.950965,0.489273,0.3731,3.8299
3980,0.109158,0.832309,0.549342,2.8445,10.5765
739,0.074011,0.293066,0.280606,-1.7291,6.9620
2659,0.008858,0.030307,0.124857,-0.4569,2.7085
1085,0.720246,0.228878,0.175200,1.9451,2.1547
3580,0.020003,0.969121,0.204662,-1.5539,7.2981


In [26]:
val_C1_vl=val_C1[['LS','FS','RS','Thetad','vel_L']]
val_C1_vl.sample(6)

,LS,FS,RS,Thetad,vel_L
221,0.740249,0.290636,0.180063,2.9486,0.4485
2230,0.046292,0.056755,0.078661,0.1521,6.6836
561,0.725532,0.669621,0.868707,-1.0256,2.8626
3757,0.764538,0.943960,0.756579,-3.1038,1.5493
1572,0.825975,0.043888,0.747140,-0.8659,2.4929
2732,0.011859,0.879199,0.858267,-1.4593,7.1370


Extract vel R, train and val

In [27]:
train_C1_vr=train_C1[['LS','FS','RS','Thetad','vel_R']]
train_C1_vr.sample(6)

,LS,FS,RS,Thetad,vel_R
2554,0.443635,0.075625,0.993564,1.1047,2.1464
1224,0.042435,0.525232,0.910040,3.0900,-3.5970
1852,0.022575,0.098642,0.025601,-0.2143,6.2161
3100,0.890984,0.568549,0.692792,1.8099,0.4049
2235,0.088441,0.047748,0.194365,-2.5142,11.5628
1181,0.886698,0.374696,0.602546,-1.1484,4.7905


In [28]:
val_C1_vr=val_C1[['LS','FS','RS','Thetad','vel_R']]
val_C1_vr.sample(6)

,LS,FS,RS,Thetad,vel_R
3918,0.276325,0.390279,0.921339,-2.0894,2.0768
3517,0.017002,0.044889,0.041190,0.4243,4.8808
1965,0.309759,0.424160,0.991133,-1.2011,3.1964
3246,0.488641,0.651894,0.689788,-2.5273,1.9095
443,0.351479,0.775697,0.041333,1.9013,7.9306
1116,0.384912,0.600286,0.671768,-2.6957,1.5513


To CSV

In [29]:
import os
parent_dir = 'Train_Val_Data_26'
child_dir_1 = os.path.join(parent_dir, 'Train')
child_dir_2 = os.path.join(parent_dir, 'Validation')
os.makedirs(child_dir_1, exist_ok=True)
os.makedirs(child_dir_2, exist_ok=True)

In [30]:

train_C1_vl.to_csv(os.path.join(child_dir_1, 'C1_vl_train.csv'), index=False)
val_C1_vl.to_csv(os.path.join(child_dir_2, 'C1_vl_val.csv'), index=False)


train_C1_vr.to_csv(os.path.join(child_dir_1, 'C1_vr_train.csv'), index=False)
val_C1_vr.to_csv(os.path.join(child_dir_2, 'C1_vr_val.csv'), index=False)

## No obstacle circle

### Angle Data Generation

In [31]:
HA_Data_2=[round(random.uniform(-math.pi, math.pi),4) for _ in range(3000)]
HA_Data_2=np.array(HA_Data).reshape(3000,1)
print('Angle data :',HA_Data_2[:2])

Angle data : [[ 2.9772]
 [-1.461 ]]


In [32]:
Df_C2=pd.DataFrame(HA_Data_2,columns=[['HA']])
Df_C2.sample(6)

,HA
819,-1.5491
809,1.4150
1136,-2.1728
245,-2.9228
2067,2.7358
10,-1.3152


### Evaluation   Right and Left Wheel

In [33]:
Lt_2=1.2
v_max=1

def lw_eval_NO(df):
    Sf1=-1
    haf=df['HA']
    Lt_pf=round((Lt_2),4)
    V_pf=round(v_max,4)
    return round((1-((LMr*haf*Sf1)/(2*Lt_pf)))*(V_pf/R),4)

def rw_eval_NO(df):
    Sf1=-1
    haf=df['HA']
    Lt_pf=round((Lt_2),4)
    V_pf=round(v_max,4)
    return round((1+((LMr*haf*Sf1)/(2*Lt_pf)))*(V_pf/R),4)


Df_C2=Df_C2.assign(W_L=Df_C2.apply(lw_eval_NO,axis=1))

Df_C2=Df_C2.assign(W_R=Df_C2.apply(rw_eval_NO,axis=1))


Df_C2.sample(6)



,HA,W_L,W_R
2478,-2.8875,2.7775,7.4789
2516,-1.1188,4.2174,6.0390
2802,0.9256,5.8817,4.3747
2207,-1.8673,3.6080,6.6484
1729,-0.7239,4.5389,5.7175
2172,0.9666,5.9151,4.3413


Left Wheel Train NO

In [34]:
Df_C2_Train=Df_C2[['HA','W_L']]
Df_C2_Train.sample(6)

,HA,W_L
2606,-1.7856,3.6745
89,2.6836,7.3129
2626,2.3897,7.0737
1870,-2.3404,3.2229
1555,2.6872,7.3159
494,-2.1422,3.3842


In [35]:
Df_C2_Train.to_csv('Df_C2_Train_L_2.csv',index=False)

Right Wheel Train NO

In [36]:
Df_C2_Train_R=Df_C2[['HA','W_R']]
Df_C2_Train_R.head(6)

,HA,W_R
0,2.9772,2.7045
1,-1.4610,6.3176
2,1.6326,3.7991
3,1.4520,3.9461
4,1.2483,4.1120
5,1.3266,4.0482


In [37]:
Df_C2_Train_R.to_csv('Df_C2_Train_R_2.csv',index=False)